In [1]:
from api.BinanceData import BinanceData
from utils.CleanData import CleanData
from variables.QuantitativeFunc import QuantitativeFunc
from variables.Target import Target
from variables.TradingIndicators import TradingIndicators
import pandas as pd

In [ ]:
# Import de données
BinanceData = BinanceData()
data = BinanceData.load_data("BTCUSDT", days=30)

# Nettoyage des données 
cleaner = CleanData()
data = cleaner.clean_klines_data(data)

# Export des données pour travailler sur un ficheir Excel plutôt que de requêter l'API Binance à chaque fois
data.to_excel("data/données_test.xlsx", index=False)

In [ ]:
# Import des données depuis le fichier Excel
data = pd.read_excel("data/données_test.xlsx")

# Création des variables rendement et la cible 
data["return"] = QuantitativeFunc.return_(data['close'])
data["return_10"] = QuantitativeFunc.return_10(data['close'])
data["target"] = Target.compute(data["return"], threshold=0.0005)

# ajout des indicateurs techniques
data = TradingIndicators.add_sma(data, price_col="close", window=20, new_col="SMA_20")
data = TradingIndicators.add_ema(data, price_col="close", window=12, new_col="EMA_12")
data = TradingIndicators.add_macd(data, price_col="close")
data = TradingIndicators.add_bollinger_bands(data, price_col="close")
data = TradingIndicators.add_rsi(data, price_col="close")
data = TradingIndicators.add_atr(data, high_col="high", low_col="low", close_col="close", window=14, new_col="ATR_14")
data = TradingIndicators.add_high_low_range(data, high_col="high", low_col="low")
data = TradingIndicators.add_buy_pressure(data, high_col="high", low_col="low", close_col="close")
data = TradingIndicators.add_realized_volatility(data, returns_col="return", window=14)

print(data.head(60))
print(len(data))
print(data.columns)
print(data["target"].value_counts())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Prix de clôture (noir, trait plein)
plt.plot(data["open_time"], data["close"], label="Close", color="black", linestyle="-", linewidth=1)

# Moyennes mobiles (pointillés)
plt.plot(data["open_time"], data["SMA_20"], label="SMA 20", color="blue", linestyle="--", linewidth=1)
plt.plot(data["open_time"], data["EMA_12"], label="EMA 12", color="orange", linestyle="--", linewidth=1)

# Bandes de Bollinger (pointillés verts)
plt.plot(data["open_time"], data["Bollinger_Upper"], label="Bollinger Upper", color="green", linestyle=":", linewidth=1)
plt.plot(data["open_time"], data["Bollinger_Lower"], label="Bollinger Lower", color="green", linestyle=":", linewidth=1)

# Remplir l'espace entre les bandes (optionnel)
plt.fill_between(data["open_time"], data["Bollinger_Lower"], data["Bollinger_Upper"], color="green", alpha=0.1)

# Légende et grille
plt.title("Prix avec SMA, EMA et Bollinger Bands (5min)")
plt.xlabel("Temps")
plt.ylabel("Prix")
plt.legend()
plt.grid(True)

plt.show()